## 동적 웹 페이지 1

In [2]:
!pip install selenium

In [10]:
from selenium import webdriver 

driver = webdriver.Chrome("chromedriver") 
driver.implicitly_wait(3)
driver.get("https://www.danawa.com/") 

login = driver.find_element_by_css_selector('li.my_page_service > a')
print("HTML 요소: ", login)
print("태그 이름: ", login.tag_name)
print("문자열: ", login.text)
print("href 속성: ", login.get_attribute('href'))

login.click()
driver.implicitly_wait(3)

my_id = "love1770"
my_pw = "m9252009$$"

driver.find_element_by_id('danawa-member-login-input-id').send_keys(my_id)
driver.implicitly_wait(2)
driver.find_element_by_name('password').send_keys(my_pw)
driver.implicitly_wait(2)
driver.find_element_by_css_selector('button.btn_login').click()

HTML 요소:  <selenium.webdriver.remote.webelement.WebElement (session="047365dbb488037337137bcd6e8521f3", element="c81ec010-c8e8-4aaf-8906-ad7249315e6c")>
태그 이름:  a
문자열:  로그인
href 속성:  https://auth.danawa.com/login?url=http%3A%2F%2Fwww.danawa.com%2F


## 동적 웹 페이지 2

In [13]:
from selenium import webdriver 

driver = webdriver.Chrome("chromedriver") 
driver.implicitly_wait(3)
driver.get("https://www.danawa.com/") 

login = driver.find_element_by_css_selector('li.my_page_service > a')
login.click()
driver.implicitly_wait(3)

my_id = "love1770"
my_pw = "m9252009$$"

driver.find_element_by_id('danawa-member-login-input-id').send_keys(my_id)
driver.implicitly_wait(2)
driver.find_element_by_name('password').send_keys(my_pw)
driver.implicitly_wait(2)
driver.find_element_by_css_selector('button.btn_login').click()
driver.implicitly_wait(2)

wishlist = driver.find_element_by_css_selector('li.interest_goods_service > a').click()
driver.implicitly_wait(2)
html_src = driver.page_source
driver.close()
# print(html_src[:])

from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(html_src, 'html.parser')
#wishProductListArea > table > tbody > tr
# wish_table = soup.select('table[class="tbl wish_tbl"]')[0]
# wish_items = wish_table.select('tbody tr')

wish_items = soup.select('#wishProductListArea > table > tbody > tr')

for item in wish_items:
    title = item.find('div', {'class':'tit'}).text
    price = item.find('span', {'class':'price'}).text
    link = item.find('a', href=re.compile('prod.danawa.com/info/')).get('href')
    print(title)
    print(price)
    print(link)
    print("\n")

삼성전자 970 EVO M.2 NVMe (500GB)
99,400원
http://prod.danawa.com/info/?pcode=6095784


삼성전자 860 EVO (500GB)
77,600원
http://prod.danawa.com/info/?pcode=5834210




## 한국은행 경제통계시스템 통계지표 활용하기 1

In [14]:
from selenium import webdriver 
import time

def download_bok_statistics():

    driver = webdriver.Chrome("chromedriver") 
    driver.implicitly_wait(3)
    driver.get("http://ecos.bok.or.kr/jsp/vis/keystat/#/key") 
    
    
    excel_download = driver.find_element_by_css_selector('img[alt="download"]')
    driver.implicitly_wait(3)
    
    excel_download.click()
    time.sleep(5)
    
    driver.close()
    print("파일 다운로드 실행...")

    return None

download_bok_statistics()

파일 다운로드 실행...


## 한국은행 경제통계시스템 통계지표 활용하기 2

In [15]:
from selenium import webdriver 
from bs4 import BeautifulSoup
import time

def download_bok_statistics_by_keyword():
    
    item_found = 0              
    while not item_found:
        
        keyword = ""       
        while len(keyword) == 0:
            keyword = str(input("검색할 항목을 입력하세요: "))
        
        driver = webdriver.Chrome("chromedriver")
        driver.implicitly_wait(3)
        driver.get("http://ecos.bok.or.kr/jsp/vis/keystat/#/key") 
        time.sleep(5)             
                   
        items1 = driver.find_elements_by_css_selector('a[class="ng-binding"]')
        items2 = driver.find_elements_by_css_selector('a[class="a-c1-list ng-binding"]')
        items3 = driver.find_elements_by_css_selector('a[class="a-c4-list ng-binding"]')
        driver.implicitly_wait(3)

        items = items1[1:] + items2 + items3    
        
        for idx, item in enumerate(items):
            if keyword in item.text:
                print("검색어 '%s'에 매칭되는 '%s' 통계지표를 검색 중..." % (keyword, item.text))
                item.click()
                item_found = 1
                time.sleep(5)
                break
            elif idx == (len(items) - 1):
                print("검색어 '%s'에 대한 통계지표가 존재하지 않습니다..." % keyword)
                driver.close()
                continue
            else:
                pass
            
    html_src = driver.page_source
    soup = BeautifulSoup(html_src, 'html.parser')
    driver.close()
  
    table_items = soup.find_all('td', {'class':'ng-binding'})
    date = [t.text for i, t in enumerate(table_items) if i % 3 == 0]
    value = [t.text for i, t in enumerate(table_items) if i % 3 == 1]    
    change = [t.text for i, t in enumerate(table_items) if i % 3 == 2]    
    
    result_file = open('bok_statistics_%s.csv' % keyword, 'w')
    
    for i in range(len(date)):
        result_file.write("%s, %s, %s" % (date[i], value[i], change[i]))
        result_file.write('\n')
    
    result_file.close()
    print("키워드 '%s'에 대한 통계지표를 저장하였습니다." % keyword)
    
    return date, value, change

result = download_bok_statistics_by_keyword()
print(result)

검색할 항목을 입력하세요: CD
검색어 'CD'에 매칭되는 'CD수익률(91일) ('20.11.20)' 통계지표를 검색 중...
키워드 'CD'에 대한 통계지표를 저장하였습니다.
(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020.4', '2020.5', '2020.6', '2020.7', '2020.8', '2020.9', '2020.10', '2020.11.13', '2020.11.16', '2020.11.17', '2020.11.18', '2020.11.19', '2020.11.20'], ['2.72', '2.49', '1.76', '1.49', '1.44', '1.68', '1.69', '1.10', '1.02', '0.79', '0.79', '0.68', '0.63', '0.63', '0.66', '0.66', '0.66', '0.66', '0.66', '0.66'], ['-0.58', '-0.23', '-0.73', '-0.27', '-0.05', '0.24', '0.01', '-0.13', '-0.08', '-0.23', '0.00', '-0.11', '-0.05', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00'])


###  대중 교통 정보

#### https://pay.tmoney.co.kr/
이용안내 클릭
대중교통 통계자료 클릭

driver.find_elements_by_xpath('//*[@id="contents"]/div[1]/table/tbody/tr[1]/td[2]/a/em')